In [90]:
import csv
import shutil
import random
import copy
import tempfile
import heapq
import pandas as pd
import numpy as np

def external_merge_sort_implementation(chunk_paths, d, index_dict, file_path):
    sorted_chunk_paths = []
    index_ls = copy.deepcopy([*index_dict.values()])
    output_len_ls = copy.deepcopy([*index_dict.values()])
    output_len_idx = 0
    output_file = f"./sorted_file"
    for i_iter, chunk_path in enumerate(chunk_paths):
        # print(f"{chunk_path = }")
        chunk_df = pd.read_pickle(chunk_path)
        if i_iter == 0:
            df_columns = chunk_df.columns
        # print(f"{chunk_df = }")
        chunk_df_ls = chunk_df.values.tolist()
        # print(f"{chunk_df_ls = }")
        chunk_df_ls.sort(key = lambda x: (x[d]))
        # print(f"{chunk_df_ls = }")
        sorted_chunk_path = f"./sorted_{chunk_path.split('/')[-1].split('.pkl')[0]}"
        sorted_chunk_df = pd.DataFrame(chunk_df_ls, columns=df_columns)
        print(f"{sorted_chunk_df = }")
        sorted_chunk_df.to_pickle(sorted_chunk_path)
        # with open(sorted_chunk_path, 'w', newline='') as sorted_chunk_file:
        #     writer = csv.writer(sorted_chunk_file)
        #     writer.writerows(chunk_df_ls)
        sorted_chunk_paths.append(sorted_chunk_path)

    # Merge sorted chunks
    output_ls = []
    idxs_ls = [0] * len(chunk_paths)
    input_ls = []
    min_idx = -1
    debug_counter = 0
    sorted_chunks_path_cpy = copy.deepcopy(sorted_chunk_paths)
    while True:
        #First iteration
        if min_idx == -1:
            for sorted_chunk_path in sorted_chunks_path_cpy:
                df = pd.read_pickle(sorted_chunk_path)
                input_ls.append(df.values.tolist()[idxs_ls[i_iter]])
        else:
            try:
                print(f"{sorted_chunks_path_cpy = }")
                print(f"{min_idx = }")
                print(f"{idxs_ls = }")
                df = pd.read_pickle(sorted_chunks_path_cpy[min_idx])
                print(f"{df = }")
                input_ls.insert(min_idx, df.values.tolist()[idxs_ls[min_idx]])
            except Exception as err:
                print(f"Inside except block... {err}")
  
        print(f"{input_ls = }")
        input_arr = np.array(input_ls)
        print(f"{input_arr = }")
        min_idx = np.argmin(input_arr, axis=0)[d]
        print(f"{min_idx = }")
        output_ls.append(input_ls[min_idx])
        print(f"{output_ls = }")
        input_ls.pop(min_idx)
        idxs_ls[min_idx] += 1
        print(f"{idxs_ls = }")

        #Writing output
        if len(output_ls) >= output_len_ls[output_len_idx]:
            print(f"{output_ls = }")
            print(f"{output_len_idx = }")
            print(f"{output_len_ls[output_len_idx] = }")
            sorted_chunk_df = pd.DataFrame(output_ls, columns=df_columns)
            sorted_chunk_df.to_pickle(f"{file_path}{iteration_num}{output_len_idx}.pkl")
            output_ls = []
            output_len_idx += 1
        
        if idxs_ls[min_idx] > (index_ls[min_idx] -1):
            print(f"Inside greater than condition...")
            idxs_ls.pop(min_idx)
            sorted_chunks_path_cpy.pop(min_idx)
            index_ls.pop(min_idx)
            if len(input_ls) == 0:
                break 
            input_ls.pop(min_idx)

In [94]:
index_dict = {}
# index_dict = {0:[0,1,2,3,4], 1:[5,6,7,8,9]}
#Randomly split into multiple chunks and test\
split_num = np.random.randint(1,11)
# split_num = 4
print(f"{split_num = }")
chunks_path = []
split_df = np.array_split(transformed_df, split_num)
for i_iter, df in enumerate(split_df):
    index_dict[i_iter] = df.shape[0]
    print(f"Original chunk: {df}")
    df.to_pickle(f"./iter_run{i_iter}.pkl")
    chunks_path.append(f"./iter_run{i_iter}.pkl")
# chunk_paths = [f"./iter_run0.pkl", f"./iter_run1.pkl"]
# sort_chunks_modif(chunk_paths, 1, index_dict)
print(index_dict)
sort_chunks_modif(chunks_path, 0, index_dict)

split_num = 3
Original chunk:    Country  Year  Sex
0        0     0    0
1        0     0    0
2        0     1    0
3        0     2    0
Original chunk:    Country  Year  Sex
4        1     2    0
5        1     3    1
6        0     1    1
Original chunk:    Country  Year  Sex
7        0     3    1
8        2     0    1
9        2     1    1
{0: 4, 1: 3, 2: 3}
sorted_chunk_df =    Country  Year  Sex
0        0     0    0
1        0     0    0
2        0     1    0
3        0     2    0
sorted_chunk_df =    Country  Year  Sex
0        0     1    1
1        1     2    0
2        1     3    1
sorted_chunk_df =    Country  Year  Sex
0        0     3    1
1        2     0    1
2        2     1    1

input_ls = [[0, 0, 0], [0, 1, 1], [0, 3, 1]]
input_arr = array([[0, 0, 0],
       [0, 1, 1],
       [0, 3, 1]])
min_idx = 0
output_ls = [[0, 0, 0]]
idxs_ls = [1, 0, 0]

sorted_chunks_path_cpy = ['./sorted_iter_run0', './sorted_iter_run1', './sorted_iter_run2']
min_idx = 0
idxs_ls = [1, 0, 0]

In [95]:
import glob
for i_iter, pkl_filename in enumerate(sorted(glob.glob("/home/vpk/courses/Data Analytics/BUC-implementation/external_merge_sort/merge_sort/fully_sorted_iter_run*"))):
    df = pd.read_pickle(pkl_filename)
    print(f"{i_iter}: {df}")

0:    Country  Year  Sex
0        0     0    0
1        0     0    0
2        0     1    0
3        0     2    0
1:    Country  Year  Sex
0        0     1    1
1        0     3    1
2        1     2    0
2:    Country  Year  Sex
0        1     3    1
1        2     0    1
2        2     1    1
3:    Country  Year  Sex
0        2     0    1
1        2     1    1


In [35]:
transformed_df

,Country,Year,Sex
0,0,0,0
1,0,0,0
2,0,1,0
3,0,2,0
4,1,2,0
5,1,3,1
6,0,1,1
7,0,3,1
8,2,0,1
9,2,1,1


In [30]:
sorted_output = [[0, 0, 0], [0, 1, 0], [0, 2, 0], [0, 1, 1], [0, 3, 1], [1, 2, 0], [1, 3, 1], [2, 0, 1], [2, 1, 1]]

In [ ]:
input_path = 'sample_preprocessed.csv'
output_path = 'sorted_sample_preprocessed.csv' 
chunk_size=9 # chunk_size=10, but the chunk list stores 11 rows, so I ma gonna make chunk_size=9.

# Step 1: Read the input data and divide into chunks
chunk_paths = []
with open(input_path, 'r') as input_file:
    reader = csv.reader(input_file)
    header = next(reader)
    print(f"{header = }")
    chunk = [header]
    count = 0
    
    print("############ 1 ############ ")
    for i_iter, row in enumerate(reader):
        # print(f"{row = }")
        chunk.append(row)
        # print(f"{chunk = }")
        count += 1
        '''
        chunk_size=10, but the chunk list stores 11 rows, so I am gonna make chunk_size=9.
        '''
        if count == chunk_size:
            chunk_path = f"./{i_iter}"
            print(f"{chunk_path = }")
            with open(chunk_path, 'w', newline='') as chunk_file:
                writer = csv.writer(chunk_file)
                writer.writerows(chunk)
            chunk_paths.append(chunk_path)
            print("############ 2 ############ ")
            # print(f"{header = }")
            # chunk = [header]
            print(f"{chunk = }")
            chunk = []
            count = 0
    print("########################## ")
        
# Step 2: Sort and merge chunks
sort_chunks(chunk_paths, output_path)

# Clean up temporary chunk files
# for chunk_path in chunk_paths:
#     os.remove(chunk_path)

In [24]:
##DataFrame for unit testing
country_name_ls = ['Albania'] *4 + ['Russia']*2 + ['Albania'] * 2 + ['India']*2
year_ls = ['2009', '2009','2010', '2011', '2011', '2012', '2010', '2012', '2009', '2010']
# sex_ls = ['m']*4 + ['f']*2 + ['m'] + ['f']*3
sex_ls = ['m']*5 + ['f']*5
test_df = pd.DataFrame()
test_df['Country'] = country_name_ls
test_df['Year'] = year_ls
test_df['Sex'] = sex_ls

In [25]:
class preprocess_df:
  '''
  Class to preprocess DataFrame
  '''
  def encode_attributes(self, input_df, column_indices):
    transformed_dicts_ls = []
    transformed_df = input_df.copy(deep = True)
    column_names = transformed_df.columns.tolist()
    # print(f"{column_names = }")
    for col_iter in column_indices:
      temp_dict = {}
      temp_key = 0
      temp_ls = []
      column_name = column_names[col_iter]
      for col in transformed_df.iloc[:,col_iter].tolist():
        if col not in [*temp_dict.keys()]:
          temp_dict[col] = temp_key
          temp_key += 1
        temp_ls.append(temp_dict[col])
      dict_inv = {v:k for k,v in temp_dict.items()}
      transformed_dicts_ls.append(dict_inv)
      transformed_df[column_name] = temp_ls
    return transformed_df, transformed_dicts_ls

In [26]:
input_df = test_df
preprocess_obj = preprocess_df()
transformed_df, column_enc_dicts_ls = preprocess_obj.encode_attributes(input_df, [*range(0,input_df.shape[1])]) #NOTE: This should be modified as required
split_df = np.array_split(transformed_df, 2) #i.e split test_df into 2 chunks
for i_iter, df in enumerate(split_df):
    df.to_pickle(f"iter_run{i_iter}.pkl")